In [2]:
import cv2
import numpy as np
import pandas as pd
import time
import autopy
import mediapipe as mp
import HandDetectionClass as HD

In [16]:
previous_time = 0
current_time = 0
cap = cv2.VideoCapture(0)
detector = HD.HandDetectionClass(min_det_con=0.65)
cap.set(3, 1280)
cap.set(4, 720)
tipsID = [4, 8, 12, 16, 20]
endID = [3, 6, 10, 14, 18]
x_prev, y_prev = -1, -1
pic_draw = np.zeros((720, 1280, 3), np.uint8)

while True:
    retval, image = cap.read()
    image = cv2.flip(image, 1)
    image = detector.findHands(image)
    landmarks_list = detector.findPosition(image, draw_indecis=False)
    if len(landmarks_list):
        x_index, y_index = landmarks_list[tipsID[1]][1], landmarks_list[tipsID[1]][2]
        x_middle, y_middle = landmarks_list[tipsID[2]][1], landmarks_list[tipsID[2]][2]
        #We will draw when 1 finger is up *index to be precise*, select when 2 fingers are up
        fingers = detector.fingers_up()
        #print(fingers)
        erase_color = (0, 0, 0)
        draw_color = (250, 250, 0)
        
            
        #Draw Mode, We can draw with index
        if fingers[1] == 1 and fingers[2] == 0:
            cv2.circle(image, (x_index, y_index), 15, (150, 100, 0), -1)
            if x_prev == -1 and y_prev == -1:
                x_prev, y_prev = x_index, y_index
            
            cv2.line(image, (x_prev, y_prev), (x_index, y_index), draw_color, 10)
            cv2.line(pic_draw, (x_prev, y_prev), (x_index, y_index), draw_color, 10)
        #Erasing using index and middle finger
        
        elif fingers[1] == 1 and fingers[2] == 1:
            cv2.rectangle(image, (x_index, y_index-15), (x_middle, y_middle-15), (150, 100, 150), -1)
            cv2.line(image, (x_prev, y_prev), (x_index, y_index), erase_color, 20)
            cv2.line(pic_draw, (x_prev, y_prev), (x_index, y_index), erase_color, 20)
        x_prev, y_prev = x_index, y_index
    
    
    imgGray = cv2.cvtColor(pic_draw, cv2.COLOR_BGR2GRAY)
    retval2, imginv = cv2.threshold(imgGray, 50, 255, cv2.THRESH_BINARY_INV)
    imginv = cv2.cvtColor(imginv, cv2.COLOR_GRAY2BGR)
    image = cv2.bitwise_and(image, imginv)
    image = cv2.bitwise_or(image, pic_draw)
    
    
    current_time = time.time()
    fps = 1/(current_time-previous_time)
    previous_time = current_time
    cv2.putText(image, str(int(fps)), (5,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)        
    #image = cv2.addWeighted(image, 1, pic_draw, 1, 0)
    cv2.imshow('Live Drawing', image)
    if cv2.waitKey(5) & 0xFF == 27:
        break
    
cap.release()
cv2.destroyAllWindows()